# Bài tập 1

### Hãy huấn luyện và dự đoán đầu ra của dữ liệu trên cả hai tập, đánh giá độ chính xác của mô hình thông qua các độ đo R2(R – squared) và MSE.

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Đọc dữ liệu
data = pd.read_csv("/Users/nguyencongtri/PycharmProjects/MachineLearning/Lab9/SAT_GPA.csv")
X = data[["SAT"]].values
y = data[["GPA"]].values

# Chuẩn hóa dữ liệu
scaler_X = StandardScaler()
scaler_y = StandardScaler()
X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y)

# Chia dữ liệu
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y_scaled, train_size=54, random_state=42)

# Chuyển dữ liệu thành dạng ma trận
X_train = X_train.T  
y_train = y_train.T  
X_val = X_val.T      
y_val = y_val.T      

In [3]:

def relu(Z):
    return np.maximum(0, Z)
def relu_deriv(Z):
    return Z > 0
def linear(Z):
    return Z

def mse_loss(Y, Yhat):
    return np.mean((Y - Yhat) ** 2)

def mse_loss_deriv(Y, Yhat):
    return Yhat - Y  

In [4]:

def initialize_parameters(input_size, hidden_size, output_size):
    np.random.seed(42)
    W1 = np.random.randn(hidden_size, input_size) * 0.01
    b1 = np.zeros((hidden_size, 1))
    W2 = np.random.randn(output_size, hidden_size) * 0.01
    b2 = np.zeros((output_size, 1))
    return W1, b1, W2, b2

def forward_propagation(X, W1, b1, W2, b2):
    Z1 = W1 @ X + b1
    A1 = relu(Z1)
    Z2 = W2 @ A1 + b2
    A2 = linear(Z2)  
    return Z1, A1, Z2, A2


def backward_propagation(X, Y, Z1, A1, Z2, A2, W1, W2):
    m = X.shape[1]
    dZ2 = mse_loss_deriv(Y, A2)
    dW2 = (1/m) * (dZ2 @ A1.T)
    db2 = (1/m) * np.sum(dZ2, axis=1, keepdims=True)
    dA1 = W2.T @ dZ2
    dZ1 = dA1 * relu_deriv(Z1)
    dW1 = (1/m) * (dZ1 @ X.T)
    db1 = (1/m) * np.sum(dZ1, axis=1, keepdims=True)
    return dW1, db1, dW2, db2

def update_parameters(W1, b1, W2, b2, dW1, db1, dW2, db2, learning_rate):
    W1 -= learning_rate * dW1
    b1 -= learning_rate * db1
    W2 -= learning_rate * dW2
    b2 -= learning_rate * db2
    return W1, b1, W2, b2

In [5]:

def train_model(X, Y, hidden_size, num_iterations, learning_rate):
    input_size = X.shape[0]
    output_size = Y.shape[0]
    W1, b1, W2, b2 = initialize_parameters(input_size, hidden_size, output_size)
    
    for i in range(num_iterations):
        Z1, A1, Z2, A2 = forward_propagation(X, W1, b1, W2, b2)
        loss = mse_loss(Y, A2)
        dW1, db1, dW2, db2 = backward_propagation(X, Y, Z1, A1, Z2, A2, W1, W2)
        W1, b1, W2, b2 = update_parameters(W1, b1, W2, b2, dW1, db1, dW2, db2, learning_rate)
        
        if i % 1000 == 0:
            print(f"Iteration {i}, Loss: {loss}")
    
    return W1, b1, W2, b2

# Dự đoán
def predict(X, W1, b1, W2, b2):
    _, _, _, A2 = forward_propagation(X, W1, b1, W2, b2)
    return A2

In [6]:

hidden_size = 10
num_iterations = 5000
learning_rate = 0.01
W1, b1, W2, b2 = train_model(X_train, y_train, hidden_size, num_iterations, learning_rate)
y_train_pred_scaled = predict(X_train, W1, b1, W2, b2)
y_val_pred_scaled = predict(X_val, W1, b1, W2, b2)
y_train_pred = scaler_y.inverse_transform(y_train_pred_scaled.T).T
y_val_pred = scaler_y.inverse_transform(y_val_pred_scaled.T).T
y_train_true = scaler_y.inverse_transform(y_train.T).T
y_val_true = scaler_y.inverse_transform(y_val.T).T

# Đánh giá mô hình
train_mse = mean_squared_error(y_train_true.T, y_train_pred.T)
val_mse = mean_squared_error(y_val_true.T, y_val_pred.T)
train_r2 = r2_score(y_train_true.T, y_train_pred.T)
val_r2 = r2_score(y_val_true.T, y_val_pred.T)

print("\nKết quả đánh giá:")
print(f"Tập huấn luyện - MSE: {train_mse:.4f}, R²: {train_r2:.4f}")
print(f"Tập validation - MSE: {val_mse:.4f}, R²: {val_r2:.4f}")

Iteration 0, Loss: 0.9861818279013259
Iteration 1000, Loss: 0.7462157824761696
Iteration 2000, Loss: 0.5771174705382282
Iteration 3000, Loss: 0.5761572232833406
Iteration 4000, Loss: 0.5754659635207152

Kết quả đánh giá:
Tập huấn luyện - MSE: 0.0419, R²: 0.4134
Tập validation - MSE: 0.0451, R²: 0.3851


### Sử dụng mô hình hồi quy tuyến tính để thực hiện lại dự đoán. So sánh với mô hình ANN trên các tiêu chí: Thời gian training; Thời gian predict (tính trung bình); độ chính xác.

In [7]:
import time
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [8]:
start_train_lr = time.time()
model_lr = LinearRegression()
model_lr.fit(X_train.T, y_train.T) 
end_train_lr = time.time()
train_time_lr = end_train_lr - start_train_lr

# Dự đoán và đo thời gian trung bình
start_pred_train_lr = time.time()
y_train_pred_scaled_lr = model_lr.predict(X_train.T)
end_pred_train_lr = time.time()
pred_time_train_lr = (end_pred_train_lr - start_pred_train_lr) / X_train.shape[1]

start_pred_val_lr = time.time()
y_val_pred_scaled_lr = model_lr.predict(X_val.T)
end_pred_val_lr = time.time()
pred_time_val_lr = (end_pred_val_lr - start_pred_val_lr) / X_val.shape[1]
avg_pred_time_lr = (pred_time_train_lr + pred_time_val_lr) / 2

# Chuyển ngược về thang gốc
y_train_pred_lr = scaler_y.inverse_transform(y_train_pred_scaled_lr)
y_val_pred_lr = scaler_y.inverse_transform(y_val_pred_scaled_lr)
y_train_true_lr = scaler_y.inverse_transform(y_train.T)
y_val_true_lr = scaler_y.inverse_transform(y_val.T)

# Đánh giá mô hình hồi quy tuyến tính
train_mse_lr = mean_squared_error(y_train_true_lr, y_train_pred_lr)
val_mse_lr = mean_squared_error(y_val_true_lr, y_val_pred_lr)
train_r2_lr = r2_score(y_train_true_lr, y_train_pred_lr)
val_r2_lr = r2_score(y_val_true_lr, y_val_pred_lr)

# In kết quả hồi quy tuyến tính
print("\nHồi quy tuyến tính:")
print(f"Thời gian huấn luyện: {train_time_lr:.6f} giây")
print(f"Thời gian dự đoán trung bình: {avg_pred_time_lr:.6f} giây/mẫu")
print(f"Tập huấn luyện - MSE: {train_mse_lr:.4f}, R²: {train_r2_lr:.4f}")
print(f"Tập validation - MSE: {val_mse_lr:.4f}, R²: {val_r2_lr:.4f}")


Hồi quy tuyến tính:
Thời gian huấn luyện: 0.005475 giây
Thời gian dự đoán trung bình: 0.000005 giây/mẫu
Tập huấn luyện - MSE: 0.0434, R²: 0.3927
Tập validation - MSE: 0.0455, R²: 0.3801


### Thay đổi số chiều layer ẩn lần lượt là 75, 50. Thực nghiệm lại và đánh giá sự thay đổi kết quả so với các thông số trong đoạn code đã cho. Hãy cho nhận xét về mối liên hệ giữa siêu tham số trên với kết quả dự đoán.

In [9]:
# Thực nghiệm với các hidden_size khác nhau
hidden_sizes = [75, 50]
results = {}

for hidden_size in hidden_sizes:
    print(f"\nThực nghiệm với hidden_size = {hidden_size}")
    # Huấn luyện mô hình
    W1, b1, W2, b2 = train_model(X_train, y_train, hidden_size, num_iterations=5000, learning_rate=0.01)
    
    # Dự đoán
    y_train_pred_scaled = predict(X_train, W1, b1, W2, b2)
    y_val_pred_scaled = predict(X_val, W1, b1, W2, b2)
    
    # Chuyển ngược về thang gốc
    y_train_pred = scaler_y.inverse_transform(y_train_pred_scaled.T).T
    y_val_pred = scaler_y.inverse_transform(y_val_pred_scaled.T).T
    y_train_true = scaler_y.inverse_transform(y_train.T).T
    y_val_true = scaler_y.inverse_transform(y_val.T).T
    
    # Đánh giá
    train_mse = mean_squared_error(y_train_true.T, y_train_pred.T)
    val_mse = mean_squared_error(y_val_true.T, y_val_pred.T)
    train_r2 = r2_score(y_train_true.T, y_train_pred.T)
    val_r2 = r2_score(y_val_true.T, y_val_pred.T)
    
    results[hidden_size] = {
        "train_mse": train_mse,
        "val_mse": val_mse,
        "train_r2": train_r2,
        "val_r2": val_r2
    }
    
    print(f"Tập huấn luyện - MSE: {train_mse:.4f}, R²: {train_r2:.4f}")
    print(f"Tập validation - MSE: {val_mse:.4f}, R²: {val_r2:.4f}")

# In kết quả so sánh với hidden_size = 10 (ban đầu)
print("\nSo sánh với hidden_size = 10 (ban đầu):")
print(f"Tập huấn luyện - MSE: 0.0419, R²: 0.4134")
print(f"Tập validation - MSE: 0.0451, R²: 0.3851")


Thực nghiệm với hidden_size = 75
Iteration 0, Loss: 0.9862054622270208
Iteration 1000, Loss: 0.5839335369276969
Iteration 2000, Loss: 0.5750694647993834
Iteration 3000, Loss: 0.5715998684015645
Iteration 4000, Loss: 0.567256653127141
Tập huấn luyện - MSE: 0.0413, R²: 0.4218
Tập validation - MSE: 0.0448, R²: 0.3899

Thực nghiệm với hidden_size = 50
Iteration 0, Loss: 0.9856438931817557
Iteration 1000, Loss: 0.5893380137905556
Iteration 2000, Loss: 0.5758975677441674
Iteration 3000, Loss: 0.5751185723617569
Iteration 4000, Loss: 0.5736131098550225
Tập huấn luyện - MSE: 0.0416, R²: 0.4182
Tập validation - MSE: 0.0450, R²: 0.3862

So sánh với hidden_size = 10 (ban đầu):
Tập huấn luyện - MSE: 0.0419, R²: 0.4134
Tập validation - MSE: 0.0451, R²: 0.3851


# Bài tập 2

In [10]:
# Đọc dữ liệu
data = pd.read_csv("/Users/nguyencongtri/PycharmProjects/MachineLearning/Lab9/vidu4_lin_reg (1).txt", delim_whitespace=True)

X = data[["TUOI", "BMI", "HA", "GLUCOSE", "CHOLESTEROL"]].values
y = data[["BEDAYNTM"]].values

# Chuẩn hóa dữ liệu
scaler_X = StandardScaler()
scaler_y = StandardScaler()
X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y)

# Chia dữ liệu: 80 dòng đầu cho train, 20 dòng sau cho test
X_train = X_scaled[:80]
y_train = y_scaled[:80]
X_test = X_scaled[80:]
y_test = y_scaled[80:]

# Chuyển dữ liệu thành dạng ma trận
X_train = X_train.T  
y_train = y_train.T  
X_test = X_test.T   
y_test = y_test.T   

In [11]:
# Huấn luyện mô hình
hidden_size = 10
num_iterations = 5000
learning_rate = 0.01
W1, b1, W2, b2 = train_model(X_train, y_train, hidden_size, num_iterations, learning_rate)

# Dự đoán
y_train_pred_scaled = predict(X_train, W1, b1, W2, b2)
y_test_pred_scaled = predict(X_test, W1, b1, W2, b2)

# Chuyển ngược về thang gốc
y_train_pred = scaler_y.inverse_transform(y_train_pred_scaled.T).T
y_test_pred = scaler_y.inverse_transform(y_test_pred_scaled.T).T
y_train_true = scaler_y.inverse_transform(y_train.T).T
y_test_true = scaler_y.inverse_transform(y_test.T).T

# Đánh giá
train_mse = mean_squared_error(y_train_true.T, y_train_pred.T)
test_mse = mean_squared_error(y_test_true.T, y_test_pred.T)
train_r2 = r2_score(y_train_true.T, y_train_pred.T)
test_r2 = r2_score(y_test_true.T, y_test_pred.T)

print("\nKết quả đánh giá:")
print(f"Tập huấn luyện - MSE: {train_mse:.4f}, R²: {train_r2:.4f}")
print(f"Tập kiểm tra - MSE: {test_mse:.4f}, R²: {test_r2:.4f}")

Iteration 0, Loss: 0.799433706809961
Iteration 1000, Loss: 0.7829449786437158
Iteration 2000, Loss: 0.6329425455903255
Iteration 3000, Loss: 0.5728352212083783
Iteration 4000, Loss: 0.5349085896179998

Kết quả đánh giá:
Tập huấn luyện - MSE: 0.0836, R²: 0.3517
Tập kiểm tra - MSE: 0.2361, R²: 0.1846


### So sánh kết quả này với kết quả phương pháp hồi quy tuyến tính

In [12]:
# Huấn luyện mô hình hồi quy tuyến tính
start_train_lr = time.time()
model_lr = LinearRegression()
model_lr.fit(X_train.T, y_train.T)  # Chuyển về dạng (n_samples, n_features)
end_train_lr = time.time()
train_time_lr = end_train_lr - start_train_lr

# Dự đoán
y_train_pred_scaled_lr = model_lr.predict(X_train.T)
y_test_pred_scaled_lr = model_lr.predict(X_test.T)

# Chuyển ngược về thang gốc
y_train_pred_lr = scaler_y.inverse_transform(y_train_pred_scaled_lr)
y_test_pred_lr = scaler_y.inverse_transform(y_test_pred_scaled_lr)
y_train_true_lr = scaler_y.inverse_transform(y_train.T)
y_test_true_lr = scaler_y.inverse_transform(y_test.T)

# Đánh giá mô hình hồi quy tuyến tính
train_mse_lr = mean_squared_error(y_train_true_lr, y_train_pred_lr)
test_mse_lr = mean_squared_error(y_test_true_lr, y_test_pred_lr)
train_r2_lr = r2_score(y_train_true_lr, y_train_pred_lr)
test_r2_lr = r2_score(y_test_true_lr, y_test_pred_lr)

# In kết quả hồi quy tuyến tính
print("\nHồi quy tuyến tính:")
print(f"Tập huấn luyện - MSE: {train_mse_lr:.4f}, R²: {train_r2_lr:.4f}")
print(f"Tập kiểm tra - MSE: {test_mse_lr:.4f}, R²: {test_r2_lr:.4f}")


Hồi quy tuyến tính:
Tập huấn luyện - MSE: 0.1062, R²: 0.1768
Tập kiểm tra - MSE: 0.2274, R²: 0.2145


# Bài tập 3

In [13]:

#  Đọc dữ liệu

data = pd.read_csv("/Users/nguyencongtri/PycharmProjects/MachineLearning/Lab9/real_estate.csv")

# Đổi tên cột để tiện xử lý
data = data.rename(columns={
    'X2 house age': 'X2',
    'X3 distance to the nearest MRT station': 'X3',
    'X4 number of convenience stores': 'X4',
    'X5 latitude': 'X5',
    'X6 longitude': 'X6',
    'Y house price of unit area': 'Y'
})

# Tạo X và y
# Chọn feature X2 → X6
X = data[['X2','X3','X4','X5','X6']]
y = data['Y']

# Nếu cần, đảm bảo X2 là số nguyên (năm)
X['X2'] = X['X2'].apply(lambda x: int(np.floor(x)))

# 2. Chia dữ liệu
X_train = X.iloc[:350]
y_train = y.iloc[:350]
X_val = X.iloc[350:]
y_val = y.iloc[350:]

#  Chuẩn hóa dữ liệu
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)


In [14]:
from sklearn.neural_network import MLPRegressor

#  Linear Regression

linreg = LinearRegression()
linreg.fit(X_train_scaled, y_train)
y_pred_lin = linreg.predict(X_val_scaled)
sse_lin = np.sum((y_val - y_pred_lin)**2)
print(f"Linear Regression SSE trên validation: {sse_lin:.2f}")


# ANN (MLP Regressor)

ann = MLPRegressor(hidden_layer_sizes=(50,), max_iter=1000, random_state=42)
ann.fit(X_train_scaled, y_train)
y_pred_ann = ann.predict(X_val_scaled)
sse_ann = np.sum((y_val - y_pred_ann)**2)
print(f"ANN SSE trên validation: {sse_ann:.2f}")
# So sánh kết quả
print("\n So sánh mô hình ")
if sse_ann < sse_lin:
    print("Nhận xét: ANN có dự đoán tốt hơn Linear Regression (SSE thấp hơn).")
else:
    print("Nhận xét: Linear Regression có dự đoán tốt hơn ANN (SSE thấp hơn).")


Linear Regression SSE trên validation: 4231.22
ANN SSE trên validation: 3464.31

 So sánh mô hình 
Nhận xét: ANN có dự đoán tốt hơn Linear Regression (SSE thấp hơn).
